In [1]:
from datetime import datetime, timedelta, timezone
import pandas as pd
from robo_trader import Portfolio, Interval, DummyExchange
from robo_trader.strategies import LorentzianStrategy, LorentzianSettings
from robo_trader.price_providers import YFinancePriceProvider
import warnings

# Suppress specific warning from numpy
warnings.filterwarnings("ignore", message="All-NaN slice encountered")

In [8]:
SYMBOL = "BTC-GBP"
inception_date = datetime(2000, 1, 1, tzinfo=timezone.utc)
start_date = datetime(2023, 7, 1, tzinfo=timezone.utc)
end_date = datetime(2023, 8, 1, tzinfo=timezone.utc)
interval = Interval.ONE_HOUR
period = 24

settings = LorentzianSettings(
    #neighborsCount = 8, maxBarsBack = 2000, useDynamicExits = True,
    #useEmaFilter = False, emaPeriod = 200,
    #useSmaFilter = False, smaPeriod = 200,
    #useKernelSmoothing = True, lookbackWindow = 8, relativeWeight = 8.0, regressionLevel = 25, crossoverLag = 2,
    #useVolatilityFilter = True,
    #useRegimeFilter = True, regimeThreshold = -.1,
    #useAdxFilter = False, adxThreshold = 25,
    #use_RSI = True, RSI_param1 = 14, RSI_param2 = 2,
    #use_WT = True, WT_param1 = 10, WT_param2 = 11,
    #use_CCI = True, CCI_param1 = 20, CCI_param2 = 2,
    #use_ADX = True, ADX_param1 = 20, ADX_param2 = 2,
    #use_MFI = True, MFI_param1 = 14
    neighborsCount = 8, maxBarsBack = 2000, useDynamicExits = False,
    useEmaFilter = False, emaPeriod = 200,
    useSmaFilter = False, smaPeriod = 200,
    useKernelSmoothing = False, lookbackWindow = 12, relativeWeight = 8.0, regressionLevel = 30, crossoverLag = 0,
    useVolatilityFilter = False,
    useRegimeFilter = False, regimeThreshold = -1,
    useAdxFilter = True, adxThreshold = 40,
    use_RSI = True, RSI_param1 = 28, RSI_param2 = 7,
    use_WT = True, WT_param1 = 5, WT_param2 = 11,
    use_CCI = False, CCI_param1 = 2, CCI_param2 = 2,
    use_ADX = False, ADX_param1 = 2, ADX_param2 = 2,
    use_MFI = False, MFI_param1 = 14
)

price_provider = YFinancePriceProvider()
exchange = DummyExchange({SYMBOL: 100}, 0.004)
portfolio = Portfolio(SYMBOL, exchange, inception_date = inception_date)
strategy = LorentzianStrategy(SYMBOL, price_provider, interval, portfolio, settings)

In [9]:
optimise_parameter = False
property_to_optimise = 'RSI_param2'
opt_start_value = 1
opt_end_value = 50
opt_step = 1
opt_test = opt_start_value

returns = []

while not optimise_parameter or opt_test < opt_end_value:  
  returns = strategy.backtest(start_date = start_date, end_date = end_date, period = period)

  if optimise_parameter:
    opt_test += opt_step
    setattr(prediction_settings, property_to_optimise, opt_test)
  else:
    break

In [10]:
print(f"Price Return: {returns['price_return'] * 100} %")
print(f"Portfolio Return: {returns['portfolio_return'] * 100} %")

print("\n-----------------------\n")

print("Discrete Performance for each period:")
# Create a DataFrame from the discrete returns
df_discrete_returns = pd.DataFrame(returns['discrete_returns'])
# Format the 'period_end_date' to show up to the minute
df_discrete_returns['period_end_date'] = df_discrete_returns['period_end_date'].dt.strftime('%Y-%m-%d %H:%M')
# Drop columns not needed
df_discrete_returns.drop(['current_price', 'current_portfolio_valuation'], axis=1, inplace=True)
# Convert returns to percentage
df_discrete_returns['price_return'] = df_discrete_returns['price_return'] * 100
df_discrete_returns['portfolio_return'] = df_discrete_returns['portfolio_return'] * 100
df_discrete_returns.to_csv(f"lorentzian_discrete_returns_{portfolio.symbol}_{interval.value}_{period}.csv", index=False)

print(df_discrete_returns)


Price Return: -5.046434989792852 %
Portfolio Return: -0.019266120541624333 %

-----------------------

Discrete Performance for each period:
     period_end_date  price_return  portfolio_return
0   2023-07-01 23:00      0.355137          0.000000
1   2023-07-02 23:00      0.240039          0.000000
2   2023-07-03 23:00      1.738719          0.000000
3   2023-07-04 23:00     -1.421025          0.000000
4   2023-07-05 23:00     -0.758434          0.000000
5   2023-07-06 23:00     -2.187234          0.000000
6   2023-07-07 23:00      0.586684          0.000000
7   2023-07-08 23:00     -0.142780          0.000000
8   2023-07-09 23:00     -0.409888          0.000000
9   2023-07-10 23:00      0.568106          0.000000
10  2023-07-11 23:00      0.178227          0.000000
11  2023-07-12 23:00     -1.202176          0.000000
12  2023-07-13 23:00      2.575528          0.000000
13  2023-07-14 23:00     -3.440922          0.000000
14  2023-07-15 23:00     -0.101182          0.000000
15  2023-07